In [ ]:
import cv2
import os

# 设置原始图像文件夹路径和保存路径
input_folder = r'D:\Class\CV\Task2\CampusZebra'     # 替换为你的图像文件夹路径
output_folder = r'D:\Class\CV\Task2\CampusZebra_new'      # 可选：输出文件夹，不想另存就设为 input_folder


# 创建输出文件夹（如果不同于输入）
os.makedirs(output_folder, exist_ok=True)

# 目标尺寸
target_width = 1280
target_height = 720

# 遍历所有 JPG 图像
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.jpg'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # 读取图像
        img = cv2.imread(input_path)
        if img is None:
            print(f"⚠️ 无法读取图像: {filename}")
            continue

        # 缩放图像
        resized_img = cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_AREA)

        # 保存图像
        cv2.imwrite(output_path, resized_img)

        print(f"已处理: {filename}")

print("✅ 所有图像已使用 OpenCV 缩放到 1280×720 并保存完成！")

In [ ]:
# preprocess_labels.py
import os
import glob
import numpy as np
from PIL import Image
import tqdm # 用于显示进度条，如果没安装请 pip install tqdm

# --- 配置参数 ---

# 数据集根目录 (请根据你的实际路径修改)
base_path = r"D:\Class\CV\Task2\CDSet\dataset_YOLO_format_3434"

# 输入目录 (原始 YOLO 标签)
train_label_input_dir = os.path.join(base_path, "labels", "train")
test_label_input_dir = os.path.join(base_path, "labels", "test")

# 输出目录 (生成的掩码图像) - 在 dataset_YOLO_format_3434 下创建 masks_crosswalk 目录
train_mask_output_dir = os.path.join(base_path, "masks_crosswalk", "train")
test_mask_output_dir = os.path.join(base_path, "masks_crosswalk", "test")

# --- 关键修改：降低图像尺寸以加速 ---
IMG_HEIGHT = 352  # 原为 720
IMG_WIDTH = 640   # 原为 1280
print(f"重要：将使用分辨率 {IMG_HEIGHT}x{IMG_WIDTH} 生成掩码以加速训练。")

# 目标类别 ID (我们要提取的类别)
# 0: crosswalk, 1: guide_arrows
TARGET_CLASS_ID = 0
# 在掩码图像中，目标类别用什么像素值表示 (背景固定为 0)
TARGET_MASK_VALUE = 1

# --- 核心处理函数 ---

def parse_yolo_label_to_mask_file(label_path, output_mask_path, img_height, img_width, target_class_id, target_mask_value):
    """
    解析单个 YOLO 标签文件，生成只包含目标类别的掩码图像。
    """
    # 创建一个全零的 NumPy 数组作为基础掩码 (背景)
    mask = np.zeros((img_height, img_width), dtype=np.uint8)

    try:
        # 读取 YOLO 标签文件
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                try:
                    # 解析每一行：class_id, x_center, y_center, width, height
                    parts = line.strip().split()
                    if not parts: continue # 跳过空行
                    class_id = int(parts[0])

                    # --- 只处理目标类别 ---
                    if class_id == target_class_id:
                        if len(parts) != 5:
                             print(f"警告：格式错误的行在 {label_path}: {line.strip()}，跳过此行")
                             continue
                        x_center, y_center, width, height = map(float, parts[1:])

                        # 验证坐标和尺寸是否在合理范围 (0 到 1)
                        if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                            print(f"警告：无效的归一化坐标在 {label_path}: {line.strip()}，跳过此框")
                            continue

                        # 将归一化的 YOLO 坐标转换为像素坐标
                        pixel_x_center = x_center * img_width
                        pixel_y_center = y_center * img_height
                        pixel_width = width * img_width
                        pixel_height = height * img_height

                        # 计算边界框的像素范围 (左上角和右下角)
                        x_min = int(pixel_x_center - pixel_width / 2)
                        x_max = int(pixel_x_center + pixel_width / 2)
                        y_min = int(pixel_y_center - pixel_height / 2)
                        y_max = int(pixel_y_center + pixel_height / 2)

                        # 确保边界框坐标在图像范围内 (裁剪)
                        x_min = max(0, x_min)
                        x_max = min(img_width, x_max) # 切片不包含末尾，所以用 img_width
                        y_min = max(0, y_min)
                        y_max = min(img_height, y_max) # 切片不包含末尾，所以用 img_height

                        # 在掩码数组的对应区域填充目标值
                        if x_min < x_max and y_min < y_max: # 确保不是无效的框
                             mask[y_min:y_max, x_min:x_max] = target_mask_value

                except (ValueError, IndexError) as e:
                    print(f"警告：解析行时出错 {label_path}: {line.strip()} - {e}，跳过此行")
                    continue

        # 使用 PIL 将 NumPy 数组保存为 PNG 图像
        # 确保父目录存在
        os.makedirs(os.path.dirname(output_mask_path), exist_ok=True)
        pil_mask = Image.fromarray(mask)
        pil_mask.save(output_mask_path)

    except FileNotFoundError:
        # 这个在 glob 阶段不太可能发生，但以防万一
        print(f"错误：找不到标签文件 {label_path}")
    except Exception as e:
        print(f"处理文件 {label_path} 时发生未预料的错误: {e}")

# --- 主执行函数 ---

def preprocess_labels_offline(label_input_dir, mask_output_dir, img_height, img_width, target_class_id, target_mask_value):
    """
    处理指定目录下的所有标签文件。
    """
    # 确保输出目录存在
    os.makedirs(mask_output_dir, exist_ok=True)
    print(f"\n将在 '{mask_output_dir}' 中保存生成的掩码 (尺寸: {img_height}x{img_width})...")

    # 查找所有 .txt 标签文件
    label_files = glob.glob(os.path.join(label_input_dir, "*.txt"))
    print(f"在 '{label_input_dir}' 中找到 {len(label_files)} 个标签文件。")

    if not label_files:
        print("警告：未找到任何标签文件，请检查输入目录路径。")
        return

    processed_count = 0
    # 使用 tqdm 显示进度条
    for label_path in tqdm.tqdm(label_files, desc=f"处理 {os.path.basename(label_input_dir)} 标签"):
        # 构建输出掩码文件的路径 (文件名与标签文件相同，扩展名改为 .png)
        base_filename = os.path.basename(label_path)
        mask_filename = os.path.splitext(base_filename)[0] + ".png"
        output_mask_path = os.path.join(mask_output_dir, mask_filename)

        # 调用核心函数处理单个文件
        parse_yolo_label_to_mask_file(
            label_path, output_mask_path, img_height, img_width, target_class_id, target_mask_value
        )
        processed_count += 1

    print(f"处理完成！共处理 {processed_count} 个文件。掩码已保存到 '{mask_output_dir}'")

# --- 运行预处理 ---
if __name__ == "__main__":
    print("="*20 + " 开始离线标签预处理 " + "="*20)
    print(f"目标分辨率: {IMG_HEIGHT}x{IMG_WIDTH}")
    print(f"目标类别 ID: {TARGET_CLASS_ID} (将被赋值为 {TARGET_MASK_VALUE})")

    preprocess_labels_offline(
        train_label_input_dir,
        train_mask_output_dir,
        IMG_HEIGHT,
        IMG_WIDTH,
        TARGET_CLASS_ID,
        TARGET_MASK_VALUE
    )

    preprocess_labels_offline(
        test_label_input_dir,
        test_mask_output_dir,
        IMG_HEIGHT,
        IMG_WIDTH,
        TARGET_CLASS_ID,
        TARGET_MASK_VALUE
    )

    print("\n所有预处理完成。")
    print("="*20 + " 预处理结束 " + "="*20)